In [ ]:
#learndataa learndataa learndataa learndataa learndataa learndataa learndataa learndataa learndataa

##### Project

# Selling toys on amazon.in

Objective: 
   - Create a classifier that can predict if a toy would sell on amazon.in given information such as title, about item, product description etc.

### Download data

The CSV file is available at the link below:
    
    - https://github.com/learndataa/datasets/raw/master/df_upload.csv

# Lets begin !

### Import libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import ListedColormap

import os
import re
import string
import csv

In [2]:
from sklearn import model_selection
from sklearn import preprocessing
from sklearn import linear_model
from sklearn import neighbors
from sklearn import cluster
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV
from sklearn import svm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neural_network import MLPClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.tree import ExtraTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import LinearSVC
from sklearn.model_selection import cross_val_score
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.metrics import roc_curve, roc_auc_score, auc
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.decomposition import TruncatedSVD
from sklearn.utils import resample
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.manifold import SpectralEmbedding
import xgboost as xgb

In [3]:
#import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from wordcloud import WordCloud
from bs4 import BeautifulSoup


In [4]:
# Set how many columns/rows to view from a DataFrame
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

#### Download stopwords

In [5]:
import nltk

# Download stop words
nltk.download('stopwords')

# Download WordNet (lexical database for English language)
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /Users/erv/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/erv/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

### Data

In [6]:
df = pd.read_csv("/Users/erv/Desktop/python/fun_projects/toys/df_upload.csv", sep='|', encoding='utf-8')

print(df.shape)

(8136, 20)


In [7]:
df.head(2)

,avg_cust_rev,sold_by,category,price,num_rev,L_cm,W_cm,H_cm,wt_gm,days_elapsed,flag__about_item,flag__prod_description,flag__from_mfg,star*num,title_num,about_num,prod_overview_num,cateogry_num,text,star*num_cat
0,3.5,GSV TRADE,kids_jewellery,1149.0,18.0,30.0,11.0,4.0,350.0,97.0,1,1,0,63.0,22,115,1,22,FLIPBOAT Trolley Suitcase for Girl's Kid Toy A...,1
1,4.1,fashion hub for all,kids_jewellery,344.0,19.0,NaN,NaN,NaN,NaN,NaN,1,0,0,77.9,28,117,1,28,ANNA CREATIONS® Girls Kids Cartoon Pretend Pla...,1


In [8]:
df['star*num_cat'].unique()

array([1, 2])

In [9]:
### Get sold_by list to put them in the stop list to be removed from keywords
sold_by_list = [item.split(' ') for item in df['sold_by'].str.lower().unique() if type(item)!=float]
sold_by_list = [item for item_list in sold_by_list for item in item_list]
sold_by_list = list(set(sold_by_list)) + ['®', '™']
print('len(sold_by_list):',len(sold_by_list))

df = df.drop(['sold_by'], axis=1)

print(df.shape)

len(sold_by_list): 2049
(8136, 19)


In [10]:
df.head(2)

,avg_cust_rev,category,price,num_rev,L_cm,W_cm,H_cm,wt_gm,days_elapsed,flag__about_item,flag__prod_description,flag__from_mfg,star*num,title_num,about_num,prod_overview_num,cateogry_num,text,star*num_cat
0,3.5,kids_jewellery,1149.0,18.0,30.0,11.0,4.0,350.0,97.0,1,1,0,63.0,22,115,1,22,FLIPBOAT Trolley Suitcase for Girl's Kid Toy A...,1
1,4.1,kids_jewellery,344.0,19.0,NaN,NaN,NaN,NaN,NaN,1,0,0,77.9,28,117,1,28,ANNA CREATIONS® Girls Kids Cartoon Pretend Pla...,1


In [11]:
df.groupby(['star*num_cat'])['star*num_cat'].count()

star*num_cat
1    6790
2    1346
Name: star*num_cat, dtype: int64

### X, y

    * 1 if (star*num<2000) else  # will NOT sell
    * 2 if (star*num>=2000) else      # should sell

In [12]:
X = df.drop(['avg_cust_rev','star*num', 'star*num_cat', 'num_rev'], axis=1)
y = df[['star*num_cat']]

### Stratified shuffle split

In [13]:
sss = StratifiedShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
sss.get_n_splits(X, y)
print(sss)


for train_index, test_index in sss.split(X, y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]
    

print('\n', X_train.shape, X_test.shape, y_train.shape, y_test.shape)
pd.concat([X_train, y_train], axis=1).groupby(['star*num_cat'])['star*num_cat'].count()

StratifiedShuffleSplit(n_splits=5, random_state=0, test_size=0.2,
            train_size=None)
TRAIN: [3088 2646 7007 ... 3391 5609 4856] TEST: [6119  419 3829 ... 6113  730 3383]
TRAIN: [ 973  901 7319 ...  485 3953 1289] TEST: [ 246 7438  911 ... 5757  220 6610]
TRAIN: [1009 6780 1810 ... 2878 2583 6184] TEST: [4154 6950  913 ... 1775 6600 3544]
TRAIN: [4918 5076 7621 ... 3819 7589 6073] TEST: [7323 3415  952 ... 1977 2936 5144]
TRAIN: [1982 2244 1251 ... 2082 3177 6907] TEST: [ 556 3175 3315 ... 5809 1098 3259]

 (6508, 15) (1628, 15) (6508, 1) (1628, 1)


star*num_cat
1    5431
2    1077
Name: star*num_cat, dtype: int64

### Clean data

In [14]:
# Text preprocessing
def prep(txt):
    txt = BeautifulSoup(txt).get_text() # remove HTML tags
    txt = re.sub("[^a-zA-Z]", " ", txt) # remove special characters
    txt = txt.lower().split() # convert to lowercase and split each word
    
    stop_w = set(stopwords.words("english")).union(set(sold_by_list)) # use a set for faster searching
    
    txt = [w for w in txt if not w in stop_w] # Remove stop words
    txt = [WordNetLemmatizer().lemmatize(w) for w in txt] # Lemmatization

    return (" ".join(txt)) # Return the words after joining each word separated by space

In [15]:
# Clean all reviews in the train set
clean_reviews = []

for i in X_train.index.values:
    txt = X_train['text'][i]
    #print('i:',i, '\n', txt, '\n', type(txt), '\n\n')
    clean_reviews.append(prep(txt))
    

In [16]:
# Clean all reviews in test set
test_reviews = []

for i in X_test.index.values:
    test_reviews.append(prep(X_test['text'][i]))

In [17]:

# Assign reviews and labels to conventional variables (X_train, y_train, X_test)
X_train_txt = np.stack(clean_reviews)
X_test_txt = np.stack(test_reviews)

In [18]:
max_features = 1000
# Use TF-IDF to vectorize
tfidf_vec = TfidfVectorizer(stop_words='english', ngram_range=(1, 2), max_features=max_features) # Range of tokens into consideration
tfidf_vec = tfidf_vec.fit(X_train_txt)

tfidf_train = tfidf_vec.transform(X_train_txt)
tfidf_test = tfidf_vec.transform(X_test_txt)


print('tfidf_train.shape:', tfidf_train.shape, ', tfidf_test.shape:', tfidf_test.shape)

tfidf_train.shape: (6508, 1000) , tfidf_test.shape: (1628, 1000)


In [19]:
# Scale
X_train_num = X_train[['price','days_elapsed','L_cm','W_cm','H_cm','wt_gm']].fillna(-1) #<-- CHECK !!!
X_test_num = X_test[['price','days_elapsed','L_cm','W_cm','H_cm','wt_gm']].fillna(-1)  #<-- CHECK !!!

scl = preprocessing.StandardScaler()
scl.fit(X_train_num)

X_train_num = scl.transform(X_train_num)
X_test_num = scl.transform(X_test_num)

print(X_train_num.shape, X_test_num.shape)

(6508, 6) (1628, 6)


In [20]:
# Binarize category
X_train_bin = pd.get_dummies(X_train[['category']], prefix=['cat'], columns = ['category'], drop_first=True)
X_test_bin = pd.get_dummies(X_test[['category']], prefix=['cat'], columns = ['category'], drop_first=True)

In [21]:
# Combine txt and num 
X_train_final = np.hstack([X_train_num, tfidf_train.toarray(), X_train_bin.to_numpy()]) #, X_train[theme_col_list].to_numpy()])
X_test_final  = np.hstack([X_test_num, tfidf_test.toarray(), X_test_bin.to_numpy()]) #, X_test[theme_col_list].to_numpy()])

print(X_train_final.shape, X_test_final.shape)

(6508, 1016) (1628, 1016)


In [22]:
y_train = y_train.to_numpy().ravel()
y_test = y_test.to_numpy().ravel()

In [23]:
X_test.head(2)

,category,price,L_cm,W_cm,H_cm,wt_gm,days_elapsed,flag__about_item,flag__prod_description,flag__from_mfg,title_num,about_num,prod_overview_num,cateogry_num,text
556,kids_jewellery,1449.0,29.7,22.8,10.2,710.0,1212.0,1,1,0,20,228,1,20,VikriDa Girl's 2 in 1 Cosmetic and Real Makeup...
3175,soft_toys,349.0,17.0,12.0,12.0,452.0,239.0,1,1,0,20,71,14,20,Urban Hub Cute Stuffed Cartoon Character Super...


### Balance dataset: SMOTE

In [24]:
from imblearn.over_sampling import SMOTE

In [25]:
print("BEFORE:", X_train_final.shape, y_train.shape)

X_train_s, y_train_s = SMOTE().fit_resample(X_train_final, y_train)

print("Type:", type(X_train_final), type(y_train))
print("AFTER:", X_train_s.shape, y_train_s.shape)

BEFORE: (6508, 1016) (6508,)
Type: <class 'numpy.ndarray'> <class 'numpy.ndarray'>
AFTER: (10862, 1016) (10862,)


### Logistic Regression WITH SMOTE

In [26]:
print('tfidf, max_features:', max_features)

# WITH SMOTE
clf = LogisticRegression(max_iter=1e4)
clf.fit(X_train_s, y_train_s)
s = clf.score(X_test_final, y_test)
print('score (with SMOTE):', s)

# WITHOUT SMOTE
clf = LogisticRegression(max_iter=1e4)
clf.fit(X_train_final, y_train)
s = clf.score(X_test_final, y_test)
print('score (without SMOTE):', s)

tfidf, max_features: 1000
score (with SMOTE): 0.8501228501228502
score (without SMOTE): 0.8832923832923832


### Gradient boost  WITH SMOTE

In [27]:
from sklearn.ensemble import GradientBoostingClassifier

In [28]:
print('tfidf, max_features:', max_features)

# WITH SMOTE
clf = GradientBoostingClassifier(n_estimators=200, random_state=0)
clf.fit(X_train_s, y_train_s)
s = clf.score(X_test_final, y_test)
print('score (with SMOTE):', s)

# WITHOUT SMOTE
clf = GradientBoostingClassifier(n_estimators=200, random_state=0)
clf.fit(X_train_final, y_train)
s = clf.score(X_test_final, y_test)
print('score (without SMOTE):', s)

tfidf, max_features: 1000
score (with SMOTE): 0.9121621621621622
score (without SMOTE): 0.9207616707616708


### Classifiers

In [29]:
classifiers = [
    LogisticRegression(max_iter=1e4),
    KNeighborsClassifier(3),
    SVC(kernel="linear", C=0.025),
    SVC(gamma=2, C=1),
    GaussianProcessClassifier(1.0 * RBF(1.0)),
    GradientBoostingClassifier(),
    DecisionTreeClassifier(max_depth=5),
    RandomForestClassifier(max_depth=5, n_estimators=10, max_features=1),
    MLPClassifier(alpha=1, max_iter=1000),
    AdaBoostClassifier(),
    GaussianNB(),
    QuadraticDiscriminantAnalysis(),
    ExtraTreeClassifier()
    
]

In [30]:
for clf in classifiers:
    clf.fit(X_train_s, y_train_s)
    clfcv = cross_val_score(clf, X_train_s, y_train_s, cv=3, scoring='accuracy')
    
    print(clf, ":", np.mean(clfcv))
    print('-'*50)

LogisticRegression(max_iter=10000.0) : 0.8777398111995557
--------------------------------------------------
KNeighborsClassifier(n_neighbors=3) : 0.8633777387177214
--------------------------------------------------
SVC(C=0.025, kernel='linear') : 0.7844782049462848
--------------------------------------------------
SVC(C=1, gamma=2) : 0.9802068504625412
--------------------------------------------------
GaussianProcessClassifier(kernel=1**2 * RBF(length_scale=1)) : 0.9675020076767251
--------------------------------------------------
GradientBoostingClassifier() : 0.910700395635649
--------------------------------------------------
DecisionTreeClassifier(max_depth=5) : 0.8032595566175008
--------------------------------------------------
RandomForestClassifier(max_depth=5, max_features=1, n_estimators=10) : 0.6972045104193209
--------------------------------------------------
MLPClassifier(alpha=1, max_iter=1000) : 0.87258446864337
--------------------------------------------------
A

/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")
/opt/anaconda3/lib/python3.8/site-packages/sklearn/discriminant_analysis.py:808: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")


QuadraticDiscriminantAnalysis() : 0.9965937901630707
--------------------------------------------------
ExtraTreeClassifier() : 0.8955078392210775
--------------------------------------------------


### Grid search CV

In [31]:
parameters = {'solver':('newton-cg', 'lbfgs', 'liblinear', 'sag', 'saga')
             }

model = LogisticRegression(max_iter = 1e4)
clf = GridSearchCV(model, parameters, scoring='accuracy', cv=10)
clf.fit(X_train_s, y_train_s)
print(clf.best_params_)
print(clf.best_score_)



{'solver': 'newton-cg'}
0.8843727392709079


In [ ]:
#learndataa learndataa learndataa learndataa learndataa learndataa learndataa learndataa learndataa